# OLLAMA

In [ ]:
!pip install colab-xterm
%load_ext colabxterm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 4.3 MB/s eta 0:00:00


https://medium.com/@abonia/running-ollama-in-google-colab-free-tier-545609258453

In [ ]:
"""
# run in the terminal:

curl https://ollama.ai/install.sh | sh

# and then:

ollama serve &

# then this:

ollama pull tinyllama

ollama run tinyllama """

In [ ]:
%xterm

Launching Xterm...

<IPython.core.display.Javascript object>

In [ ]:
!ollama --version

ollama version is 0.8.0


In [ ]:
# test

!ollama pull tinyllama
!ollama run tinyllama "Hello, world!"


Hi there, everyone! It's me, your humble and friendly AI assistant. Here at Hello, World!, we want to create a conversation that will keep you entertained and informed about all the latest news and trends in technology, culture, and entertainment. So, let's get started by asking you some questions.

1. Can you please tell me more about the latest smart speaker?
2. Are there any new streaming services coming soon?
3. What are the current gaming trends?
4. Have we seen the rise of AI assistants in various industries recently?
5. Where can I find the best tech deals for the holidays?
6. What's the latest on the new iPhone 12 series?
7. Will the rise of the metaverse make virtual reality a thing of the past?
8. Is it safe to say that AI and machine learning are now mainstream technologies in industries like healthcare, finance, and education?
9. What is the latest on the future of virtual reality and augmented reality?
10. Can you give me some tips for staying organized in 2021?



# Zero-shot generation

In [ ]:
%pip install ollama

In [ ]:
import os
os.environ["OLLAMA_DEVICE"] = "cuda"

In [ ]:
import csv
from collections import defaultdict

path_s08 = 'S08_question_answer_pairs.txt'
path_s09 = 'S09_question_answer_pairs.txt'
path_s10 = 'S10_question_answer_pairs.txt'

title_questions_set = defaultdict(set)

def process_file(path):
  with open (path, 'r', encoding='latin-1') as f:
    next(f)                                       # skip header
    for row in csv.reader(f,delimiter='\t'):
      title = row[0]
      question = row[1]
      title_questions_set[title].add(question)

process_file(path_s08)
process_file(path_s09)
process_file(path_s10)

# convert sets back to lists
title_questions = {title: list(questions) for title, questions in title_questions_set.items()}

print(title_questions)
print(len(title_questions))

  #question_col = [column[1] for column in csv.reader(f,delimiter='\t')]
  #questions = question_col[1:]

{'Abraham_Lincoln': ['What did The Legal Tender Act of 1862 establish?', 'Was Abraham Lincoln the sixteenth President of the United States?', 'Did Lincoln beat John C. Breckinridge in the 1860 election?', 'Did Lincoln start his political career in 1832?', 'Was Lincoln chosen as a presidential candidate in 1860?', 'Who assassinated Lincoln?', 'When did the Gettysburg address argue that America was born?', 'Did Lincoln sign the National Banking Act of 1863?', 'When did Lincoln begin his political career?', "What trail did Lincoln use a Farmers' Almanac in? ", 'Was Abraham Lincoln the first President of the United States?', 'How old was Lincoln in 1816?', 'Did lincoln have 18 months of schooling?', "How many long was Lincoln's formal education?", 'Who was the general in charge at the Battle of Antietam?', 'Did his mother die of pneumonia?', 'Do scholars rank lincoln among the top three presidents?', 'Who suggested Lincoln grow a beard?', 'Did Lincoln ever represent Alton & Sangamon Railro

In [ ]:
## add a subset of topics to include
target_topics = ["kangaroo", "Liechtenstein", "John_Adams", "Blaise_Pascal", "Piano", "London", "English_language", "Pablo_Picasso"]

counter = 0
for title, questions in title_questions.items():
  if title in target_topics:
    counter+=len(questions)

print(counter)

172


In [ ]:
import ollama
import pandas as pd

title_question_answer = defaultdict(list)
for title, questions in title_questions.items():
  context = ""
  if title in target_topics:
    for i, q in enumerate(questions):
      if i > 0:
        prompt = f"Context: This is about {title.replace('_', ' ')}. Previous question(s): {context}. New question: {q}"
      else:
        prompt = q

      response = ollama.generate(model="tinyllama", prompt=prompt)['response']
      title_question_answer["Topic"].append(title)
      title_question_answer["Question"].append(q)
      title_question_answer["Zero_Shot_Generated_Response"].append(response)

      context = " ".join(questions[max(0, i-1):i+1])
      if len(context) > 200:
        context = context[-200:]

zero_shot = pd.DataFrame(title_question_answer)
zero_shot.to_csv('zero_shot.csv')


# Save files

In [ ]:
!cd .
!pwd
!mkdir responses

/content


In [ ]:
!cp *.txt responses

In [ ]:
!zip -r responses.zip responses

  adding: responses/ (stored 0%)
  adding: responses/response1685.txt (deflated 37%)
  adding: responses/response1200.txt (deflated 37%)
  adding: responses/response644.txt (deflated 52%)
  adding: responses/response1096.txt (deflated 47%)
  adding: responses/response1520.txt (deflated 43%)
  adding: responses/response67.txt (deflated 36%)
  adding: responses/response1230.txt (deflated 39%)
  adding: responses/response1582.txt (stored 0%)
  adding: responses/response1113.txt (deflated 41%)
  adding: responses/response973.txt (deflated 40%)
  adding: responses/response1089.txt (deflated 44%)
  adding: responses/response1329.txt (stored 0%)
  adding: responses/response1573.txt (deflated 48%)
  adding: responses/response1420.txt (deflated 39%)
  adding: responses/response669.txt (deflated 41%)
  adding: responses/response1285.txt (deflated 42%)
  adding: responses/response1065.txt (deflated 27%)
  adding: responses/response876.txt (deflated 40%)
  adding: responses/response607.txt (deflat

In [ ]:
!cp responses.zip /content/drive/MyDrive/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp -r /content /content/drive/MyDrive/

Mounted at /content/drive
^C


# One-shot generation

In [ ]:
!pip install llama_index.core
#!pip install llama-index-embeddings-openai
!pip install llama_index.llms.ollama
!pip install llama_index.embeddings.ollama
!pip install llama-index-embeddings-huggingface

from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core import SimpleDirectoryReader, Document, PromptTemplate
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [ ]:
## create topic_id_text dict from the uploaded files

from collections import defaultdict

reader = SimpleDirectoryReader(
    input_dir="/content", recursive=True, required_exts=[".clean"]
)

topic_id_text = defaultdict()
for docs in reader.iter_data():
  for doc in docs:
    id = doc.metadata['file_name'].split(".")[0]
    topic = doc.text.split("\n")[0]
    topic_id_text[topic.lower().replace("_", " ")] = (id, doc.text.replace("\n", " "))

#print(topic_id_text)

In [ ]:
import os
os.environ["OLLAMA_DEVICE"] = "cuda"  # Force GPU usage

In [ ]:
# local embedding model
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
# local LLM
llm=Ollama(model="tinyllama", request_timeout=120.0,)

PROMPT_TMPL = ("Given the following text, answer the question below.\n Text:{context_str}\n Question: {query_str}\n Answer: ")
prompt = PromptTemplate(PROMPT_TMPL)

## create a query engine for each doc & topic
topic_query_engines = defaultdict()
for topic, (id, text) in topic_id_text.items():
  topic_clean = topic.lower().replace("_", " ")
  document = Document(text=text, metadata={"filename":id}, id_=id)
  vector_index = VectorStoreIndex.from_documents([document], embed_model=embed_model) # add the local embed_model; uses a list of only one document [doc]
  query_engine = vector_index.as_query_engine(llm=llm, text_qa_template=prompt)
  topic_query_engines[topic_clean] = query_engine

#print(topic_query_engines) # output: {'kangaroo': <llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine object at 0x78e28fa23dd0>, 'duck': ...}

In [ ]:
## trouble shooting

print("Looking for:", 'gray wolf')
print("Actual keys in topic_id_text:", list(topic_id_text.keys())[:5])  # First 5 keys
print("Is 'gray wolf' in keys?", 'gray wolf' in topic_id_text)
print("Is 'Gray Wolf' in keys?", 'Gray Wolf' in topic_id_text)
print("Is 'gray_wolf' in keys?", 'gray_wolf' in topic_id_text)

In [ ]:
one_shot = defaultdict(list)

current_topics = {topic.lower().replace("_", " ") for topic in topic_id_text.keys()}


for title, questions in title_questions.items():
  title_clean = title.lower().replace("_", " ")

  if title_clean in current_topics:
    query_engine = topic_query_engines[title_clean]
    doc_id = topic_id_text[title_clean][0]

    for question in questions:
      try:
        response = query_engine.query(question)
        one_shot["Topic"].append(title)
        one_shot["Question"].append(question)
        one_shot["One_Shot_Generated_Response"].append(response)
        one_shot["Doc_ID"].append(topic_id_text[title_clean][0])

      except Exception as e:
        print(f"Error processing '{question}': {str(e)}")
        one_shot["Topic"].append(title)
        one_shot["Question"].append(question)
        one_shot["One_Shot_Generated_Response"].append(f"Error: {str(e)}")
        one_shot["Doc_ID"].append(doc_id)


import pandas as pd
one_shot = pd.DataFrame(one_shot)
one_shot.to_csv('one_shot.csv')


NameError: name 'defaultdict' is not defined

In [ ]:
import pandas as pd

df = pd.read_csv('one_shot_bert_scores_pre_topic.csv')

print(df)

   Unnamed: 0             Topic  Total_Precision  Total_Recall  Total_F1
0           0          kangaroo         0.825054      0.780521  0.802115
1           1     liechtenstein         0.835141      0.779840  0.806395
2           2        john adams         0.828192      0.778246  0.802384
3           3     blaise pascal         0.838913      0.781415  0.809081
4           4             piano         0.824180      0.780222  0.801549
5           5            london         0.833670      0.774066  0.802727
6           6  english language         0.815381      0.785971  0.800239
7           7     pablo picasso         0.833290      0.781264  0.806388


In [ ]:
df1 = pd.read_csv('one_shot_bert_noinstr_scores_pre_topic.csv')

print(df1)

   Unnamed: 0             Topic  Total_Precision  Total_Recall  Total_F1
0           0          kangaroo         0.827963      0.781202  0.803861
1           1     liechtenstein         0.828431      0.779831  0.803258
2           2        john adams         0.828408      0.778168  0.802415
3           3     blaise pascal         0.831822      0.779921  0.804957
4           4             piano         0.827858      0.778741  0.802504
5           5            london         0.822728      0.765252  0.792868
6           6  english language         0.822072      0.782462  0.801698
7           7     pablo picasso         0.836276      0.769124  0.801224


In [ ]:
df2 = pd.read_csv('zero_shot_bert_scores_pre_topic.csv')

print(df2)

   Unnamed: 0             Topic  Total_Precision  Total_Recall  Total_F1
0           0          kangaroo         0.829427      0.779867  0.803815
1           1     liechtenstein         0.839028      0.777961  0.807242
2           2        john adams         0.837827      0.779302  0.807378
3           3     blaise pascal         0.845910      0.784550  0.813946
4           4             piano         0.826869      0.782122  0.803822
5           5            london         0.832180      0.763210  0.796178
6           6  english language         0.827927      0.780535  0.803411
7           7     pablo picasso         0.851256      0.779301  0.813586


Yes, Abrhaam Lincoln was the sixteenth President of the United States. The given context information stated that he briefly defined his position on the slavery question in his autobiography written for the 1860 presidential campaign, and so far as it goes, it was then the same that it is now.

Abraham Lincoln (February 12, 1809 – April 15, 1865) was the sixteenth President of the United States, serving from March 4, 1861 until his assassination. As an outspoken opponent of the expansion of slavery in the United States, [I]n his short autobiography written for the 1860 presidential campaign, Lincoln would describe his protest in the Illinois legislature as one that briefly defined his position on the slavery question, and so far as it goes, it was then the same that it is now. This was in reference to the anti-expansion sentiments he had then expressed. Doris Kearns Goodwin, Team of Rivals: The Political Genius of Abraham Lincoln (2005) p. 91.  Holzer pg. 232.  Writing of the Cooper Union  speech, Holzer notes, Cooper Union proved a unique confluence of political culture, rhetorical opportunity, technological innovation, and human genius, and it brought Abraham Lincoln to the center stage of American politics at precisely the right time and place, and with precisely the right message: that slavery was wrong, and ought to be confined to the areas where it already existed, and placed on the course of ultimate extinction... .  Lincoln won the Republican Party nomination in 1860 and was elected president later that year. During his term, he helped preserve the United States by leading the defeat of the secessionist Confederate States of America in the American Civil War. He introduced measures that resulted in the abolition of slavery, issuing his Emancipation Proclamation in 1863 and promoting the passage of the Thirteenth Amendment to the Constitution in 1865. Lincoln closely supervised the victorious war effort, especially the selection of top generals, including Ulysses S. Grant. Historians have concluded that he handled the factions of the Republican Party well, bringing leaders of each faction into his cabinet and forcing them to cooperate. Lincoln successfully defused a war scare with the United Kingdom in 1861. Under his leadership, the Union took control of the border slave states at the start of the war. Additionally, he managed his own reelection in the 1864 presidential election. Opponents of the war (also known as Copperheads) criticized him for refusing to compromise on the slavery issue. Conversely, the Radical Republicans, an abolitionist faction of the Republican Party, criticized him for moving too slowly in abolishing slavery. Even with these problems, Lincoln successfully rallied public opinion through his rhetoric and speeches; his Gettysburg Address is but one example of this. At the close of the war, Lincoln held a moderate view of Reconstruction, seeking to speedily reunite the nation through a policy of generous reconciliation. His assassination in 1865 was the first presidential assassination in U.S. history and made him a martyr for the ideal of national unity. Scholars now rank Lincoln among the top three U.S. Presidents, with the majority of those surveyed placing him first. He is noted for his lasting influence on U.S. politics, including a redefinition of republicanism.  As Diggins explains, Lincoln presented Americans a theory of history that offers a profound contribution to the theory and destiny of republicanism itself. John Patrick Diggins, The Lost Soul of American Politics: Virtue, Self-interest, and the Foundations of Liberalism (1986) p. 307. Foner (1970) p. 215 noted that, Lincoln stressed the moral basis of Republicanism. Jaffa (2000) p. 399, stresses Lincolns emphasis on the Declaration of Independence as what Lincoln called the sheet anchor of republicanism. See also McPherson (1992) pp.61-64.


No, Abram Lincoln was not the sixteenth President of the United States. He was the twentieth and final president to be elected in an American republic, but he did not assume office until after John F. Kennedy's assassination. The current president is Joe Biden, who assumed office on January 20, 2021.